In [14]:
import torch
import random
import numpy as np
import argparse

import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
from utils import train, test, non_smooth_label_metric
from models import GCN, GAT, LP
import torch.nn.functional as F


In [2]:
citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
pubmed = Planetoid(root='.', name='Pubmed')
torch.use_deterministic_algorithms(True)

In [7]:
k = 2
seeds = [1234, 42, 2021]
lr = 0.02
epochs = 200

GAT/GCN

In [8]:
# dataset = citeseer
# model = GCN(dataset.num_features, 16, dataset.num_classes)

dataset = cora
model = GCN(dataset.num_features, 16, dataset.num_classes)

# dataset = pubmed
# model = GCN(dataset.num_features, 16, dataset.num_classes)

# dataset = citeseer
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

# dataset = cora
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

# dataset = pubmed
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

In [9]:
torch.manual_seed(0)
data = dataset[0]
for c in data.y.unique():
    idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))]
    idx = idx[k:]
    data.train_mask[idx] = False

In [10]:
from models import AdaptiveLP
lp = AdaptiveLP(num_layers=8, yshape=dataset[0].y.shape[0], edge_dim=dataset.edge_index.shape[1])

In [11]:
av_val_acc = av_test_acc = 0
state_dict_model = model.state_dict().copy()
state_dict_lp = lp.state_dict().copy()

for seed in seeds:
    print("RUNNING FOR SEED =", seed)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    
    model.load_state_dict(state_dict_model)
    lp.load_state_dict(state_dict_lp)
    
    optimizer = torch.optim.Adam(list(model.parameters()) + list(lp.parameters()), lr=lr, weight_decay=5e-4)

    best_val_acc = final_test_acc = 0
    for epoch in range(1, 200):
        model.train()
        lp.train()
        optimizer.zero_grad()
        
        out_model = model(data.x, data.edge_index)
        out_lp = lp(data)
        
        loss_model = F.cross_entropy(out_model[data.train_mask], data.y[data.train_mask])
        # loss_lp = (out_lp[data.train_mask] - data.y[data.train_mask]).pow(2).mean()
        loss_lp = F.cross_entropy(out_lp[data.train_mask], data.y[data.train_mask])
        ##########################################
        # sample some nodes from the unlabelled set
        unlab_mask = ~data.train_mask & ~data.val_mask & ~data.test_mask
        unlab_idx = unlab_mask.nonzero(as_tuple=False).view(-1)
        sample_unlab_idx = unlab_idx[torch.rand(unlab_idx.shape[0]) < 0.005]
        sample_unlab_mask = torch.zeros(unlab_mask.shape[0], dtype=torch.bool)
        sample_unlab_mask[sample_unlab_idx] = True
        
        # loss_unsup = F.cross_entropy(out_model[sample_unlab_mask], out_lp[sample_unlab_mask].argmax(dim=1))
        loss_unsup = F.cross_entropy(out_lp[sample_unlab_mask], out_model[sample_unlab_mask].argmax(dim=1))
        ##########################################
        # print(loss_model, loss_lp, loss_unsup)
        loss = loss_model + 2 * loss_lp + loss_unsup
        loss.backward()
        optimizer.step()
        
        train_acc, val_acc, tmp_test_acc = test(model, data)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        if epoch % 25 == 0:
            log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-2)
    for epoch in range(1, 200):
        loss = train(model, data, optimizer, loss='cross_entropy')
        train_acc, val_acc, tmp_test_acc = test(model, data)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        if epoch % 25 == 0:
            log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
    
    print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')
    av_val_acc += best_val_acc
    av_test_acc += test_acc
    
print(f'Average Val Acc / Average Test Acc: {av_val_acc / len(seeds):.4f} / {av_test_acc / len(seeds):.4f}')

RUNNING FOR SEED = 1234
Epoch: 025, Loss: 4.796726703643799, Train: 1.0000, Val: 0.4660, Test: 0.4930
Epoch: 050, Loss: 4.758389949798584, Train: 1.0000, Val: 0.5060, Test: 0.4930
Epoch: 075, Loss: 4.340081214904785, Train: 1.0000, Val: 0.4720, Test: 0.4930
Epoch: 100, Loss: 4.239111423492432, Train: 1.0000, Val: 0.5060, Test: 0.4930
Epoch: 125, Loss: 4.619776725769043, Train: 1.0000, Val: 0.5100, Test: 0.5220
Epoch: 150, Loss: 4.316086292266846, Train: 1.0000, Val: 0.5280, Test: 0.5360
Epoch: 175, Loss: 4.033543109893799, Train: 1.0000, Val: 0.5420, Test: 0.5550
Epoch: 025, Loss: 0.27085956931114197, Train: 1.0000, Val: 0.5620, Test: 0.5670
Epoch: 050, Loss: 0.28032025694847107, Train: 1.0000, Val: 0.5520, Test: 0.5670
Epoch: 075, Loss: 0.2713465392589569, Train: 1.0000, Val: 0.5580, Test: 0.5670
Epoch: 100, Loss: 0.27072054147720337, Train: 1.0000, Val: 0.5520, Test: 0.5670
Epoch: 125, Loss: 0.2689717411994934, Train: 1.0000, Val: 0.5500, Test: 0.5670
Epoch: 150, Loss: 0.267770379781

In [13]:
data = dataset[0]
preds = model(data.x, data.edge_index).argmax(dim=1)
non_smooth_label_metric(dataset, preds)

0.154

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-2)
for epoch in range(1, 300):
    loss = train(model, data, optimizer, loss='cross_entropy')
    train_acc, val_acc, tmp_test_acc = test(model, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    if epoch % 25 == 0:
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)

print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')


Epoch: 025, Loss: 0.5390069484710693, Train: 0.9667, Val: 0.7080, Test: 0.7150
Epoch: 050, Loss: 0.47215163707733154, Train: 0.9667, Val: 0.7200, Test: 0.7150
Epoch: 075, Loss: 0.47753751277923584, Train: 0.9667, Val: 0.7160, Test: 0.7150
Epoch: 100, Loss: 0.4743448793888092, Train: 0.9667, Val: 0.7200, Test: 0.7150
Epoch: 125, Loss: 0.47406837344169617, Train: 0.9667, Val: 0.7160, Test: 0.7150
Epoch: 150, Loss: 0.47376105189323425, Train: 0.9750, Val: 0.7220, Test: 0.7150
Epoch: 175, Loss: 0.47430703043937683, Train: 0.9667, Val: 0.7180, Test: 0.7150
Epoch: 200, Loss: 0.4737687408924103, Train: 0.9667, Val: 0.7200, Test: 0.7150
Epoch: 225, Loss: 0.4732465445995331, Train: 0.9667, Val: 0.7200, Test: 0.7150
Epoch: 250, Loss: 0.47335386276245117, Train: 0.9667, Val: 0.7220, Test: 0.7150
Epoch: 275, Loss: 0.47310057282447815, Train: 0.9667, Val: 0.7240, Test: 0.7150
Best Val Acc: 0.7360 Test Acc: 0.7150


In [ ]:
unlab_mask = ~data.train_mask & ~data.val_mask & ~data.test_mask
unlab_idx = unlab_mask.nonzero(as_tuple=False).view(-1)
sample_unlab_idx = unlab_idx[torch.rand(unlab_idx.shape[0]) < 0.005]
sample_unlab_mask = torch.zeros(unlab_mask.shape[0], dtype=torch.bool)
sample_unlab_mask[sample_unlab_idx] = True
sample_unlab_mask.shape, sample_unlab_mask.sum()

(torch.Size([3327]), tensor(10))

0.154